# Mashup datasets

This Jupyter Notebook analyses the mashup datasets for ["Blessed Be the Fruit"](https://github.com/OrsolaMBorrini/blessedfruit), an open data project regarding the analysis of factors that might influence pregnancy rates in young women in Italy.

From the starting 7 source datasets, three different mashup datasets have been created:
Id | Dataset | Description (factor of interest) | Original source datasets
--- | --- | --- | --- 
MD1 | Religious observance in each region | RELIGION - % of religious observance in each region (over the total population) | D1, D2, D3 
MD2 | Pregnancy rates in young women in each region | PREGNANCY - % of pregnancies in young women (15-25) in each region (over the total population of young women aged 15-25) | D4, D5, D6
MD3 | (Higher) education rates in young women in each region | EDUCATION - % of women early leavers (18-24) in each region (over the total population) | D1, D2, D7

### Disclaimer
This Jupyter Notebook is of informational nature only, it is not thought to be used for the clean up process for the datasets, but only for their analysis and the explanation of the process. The code actually used for the clean up can be found in `scripts > MASHUP.py`.


## Mashup phase
### MD1 - Religious observance in each region
To create a dataset showing the percentage of religious observance in each region, we need to compare the total general population of each region (with no distinction of gender) to the religious observance (in thousands value) in each region. To get this result we will use the following source datasets:
* D1 and D2 -> for the data regarding total general population in each region for each of the three years of interest (`data > cleanDS > Population2017_Generalclean.csv`, `Population2018_Generalclean.csv`, `Population2019_Generalclean.csv`)
* D3 -> for the data regarding religious observance (thousands value) in each region (`data > cleanDS > D3_clean.csv`)

First of all the datasets regarding general population were stripped of any distinction between genders and the various ages were merged together. Additional columns to keep track of the time were added accordingly.

In [14]:
import csv
from pandas import *
import functools as ft

# ==== MD1 ====

# GENERAL POPULATION FILES
pop17 = read_csv("../data/cleanDS/Population2017General_clean.csv")
pop17 = pop17.drop(pop17[(pop17.Sex == "Males") | (pop17.Sex == "Females")].index)
gen_pop17 = (pop17.groupby(["Region code","Region"])["Population"].sum()).reset_index()
gen_pop17["Time"] = 2017

def generalPop(path):
    file = read_csv(path)
    file.drop(["Males","Females"],axis=1,inplace=True)
    file = (file.groupby(["ITTER107","Region"])["Population"].sum()).reset_index()
    return file

pop18 = "../data/cleanDS/Population2018General_clean.csv"
pop19 = "../data/cleanDS/Population2019General_clean.csv"

gen_pop18 = generalPop(pop18)
gen_pop18["Time"] = 2018
gen_pop19 = generalPop(pop19)
gen_pop19["Time"] = 2019

# Example
gen_pop19

,ITTER107,Region,Population,Time
0,ITC1,Piemonte,8622434,2019
1,ITC2,Valle d'Aosta / Vallée d'Aoste,250068,2019
2,ITC3,Liguria,3049652,2019
3,ITC4,Lombardia,20055204,2019
4,ITD3,Veneto,9758266,2019
5,ITD4,Friuli-Venezia Giulia,2412432,2019
6,ITD5,Emilia-Romagna,8928238,2019
7,ITDA,Trentino Alto Adige / Südtirol,2156138,2019
8,ITE1,Toscana,7385110,2019
9,ITE2,Umbria,1740330,2019


Two ancillary functions were created to first get the percentage values from the thousands values that we had in the tables (function `getPercentage`) and to add the column with the percentage value in the specific dataframe (`addPercentageColumn`).

In [15]:
d3_clean = read_csv("../data/cleanDS/D3_clean.csv")

def getPercentage(num,pop):
    return num * 100 / pop

def addPercentageColumn(yearlyDF):
    for idx,row in yearlyDF.iterrows():
        perc = getPercentage((row["Value"]*1000),row["Population"])
        yearlyDF.loc[idx,"Percentage"] = perc
    return yearlyDF

As the process is very similar with regards for each year, we have created one single function to create (and then save as a csv file) all three mashup datasets: `createYearlyDF`.

This function has two input parameters:
1. `genPopList` -> a list of the general population dataframes
2. `rightDF` -> the dataframe regarding religious observance (containing data for <b>all three years</b>, from 2017 to 2019)

For each year (which is obtained by checking the first value under the column "Time" of whatever general population dataframe), we cleaned up the religious observance dataframe, keeping only the data needed for that specific year (see: if-elif structure in the code below + drop instruction).

Then, we have merged the two input dataframes together using the unique code region and we subsequently dropped other unnecessary and repeated columns (namely "Territory","TIME"...).

Finally, we called the ancillary function to add a "Percentage" column to the resulting dataframe and we have saved it as a new csv file (this last step has been commented out after a first run in order to avoid interfeering with the rest of the code/project).


In [16]:
def createYearlyDF(genPopList,rightDF):
    for item in genPopList:
        if item["Time"][0] == 2017:
            d3_2017 = rightDF.drop(rightDF[(rightDF.TIME == 2018) | (rightDF.TIME == 2019)].index)
            MD1_17 = (merge(item,d3_2017,left_on="Region code",right_on="ITTER107")).drop(["Territory","ITTER107","TIME"],axis=1)
            addPercentageColumn(MD1_17)
            #MD1_17.to_csv("data/mashupDS/MD1_17.csv")
        elif item["Time"][0] == 2018:
            d3_2018 = rightDF.drop(rightDF[(rightDF.TIME == 2017) | (rightDF.TIME == 2019)].index)
            MD1_18 = (merge(item,d3_2018,left_on="ITTER107",right_on="ITTER107")).drop(["Territory","TIME"],axis=1)
            addPercentageColumn(MD1_18)
            #MD1_18.to_csv("data/mashupDS/MD1_18.csv")
        elif item["Time"][0] == 2019:
            d3_2019 = rightDF.drop(rightDF[(rightDF.TIME == 2017) | (rightDF.TIME == 2018)].index)
            MD1_19 = (merge(item,d3_2019,left_on="ITTER107",right_on="ITTER107")).drop(["Territory","TIME"],axis=1)
            addPercentageColumn(MD1_19)
            #MD1_19.to_csv("data/mashupDS/MD1_19.csv")
    return        

createYearlyDF([gen_pop17,gen_pop18,gen_pop19],d3_clean)

In the end, we have <b>three mashup datasets</b> (one for each year in our scope) regarding <b>religious observance rates in each region in Italy</b>.

In [17]:
md1_17 = read_csv("../data/mashupDS/MD1_17.csv")
md1_17

,Unnamed: 0,Region code,Region,Population,Time,TIPO_DATO_AVQ,Value,Percentage
0,0,ITC1,Piemonte,4349911,2017,6_NEVER_RELIG,1105,25.402819
1,1,ITC1,Piemonte,4349911,2017,6_WEEK_RELIG,1007,23.149899
2,2,ITC2,Valle D'Aosta/Vallée D'Aoste,126213,2017,6_NEVER_RELIG,37,29.315522
3,3,ITC2,Valle D'Aosta/Vallée D'Aoste,126213,2017,6_WEEK_RELIG,22,17.430851
4,4,ITC3,Liguria,1541541,2017,6_WEEK_RELIG,252,16.347278
5,5,ITC3,Liguria,1541541,2017,6_NEVER_RELIG,536,34.770402
6,6,ITC4,Lombardia,9986962,2017,6_WEEK_RELIG,2539,25.423147
7,7,ITC4,Lombardia,9986962,2017,6_NEVER_RELIG,2586,25.893760
8,8,ITD3,Veneto,4880936,2017,6_WEEK_RELIG,1302,26.675211
9,9,ITD3,Veneto,4880936,2017,6_NEVER_RELIG,964,19.750310


### MD2 - Pregnancy rates in young women in each region

#### MD2 - Absolute values

This mashup dataset contains the <b>total number of pregnancies</b> in young women aged 15-25, divided per region. A dataset for each year (2017, 2018, 2019) has been created.

The following source datasets have been used:
* D4 - "Mother - Age and Citizenship"
* D5 -  "Spontaneous abortions - resignation from the place of the event: Age of women - prov."
* D6 - "Induced abortions - Migration: Events by region of residence of the woman and region of intervention"

The result dataset contains the following columns:
<table>
<tr>
<th>ITTER107</th>
<th>Region</th>
<th>Live_births</th>
<th>Miscarriages</th>
<th>Abortions</th>
<th>Total</th>
<th>Time</th>
</tr>
</table>

<code>Total</code> contains the data retreived by summing values of live_births, spontaneous abortions and induced abortions in D4, D5, D6.

<b>Disclaimer</b>

* Although our selected age group would be 15-25 years old, unfortunately, these istat datasets were organized in age classes of 15-24, 25-29. For this reason we have decided to keep <b>only age class 15-24</b> since, including pregnancies data for women closer to 30 years old, hence closer to the average age of pregnancy in women, would have falsified our data too much

* <span style="color: blue;">Dataset 5</span> does not have values for a class age of 15 -24 as the other two datasets have, instead, it gives values of live births aggregated for a class of women <span style="color: blue;">until 17 years old</span>. However, we considered valid to aggregate numbers from this dataset without further distinction of age, since values for pregnancies under 15 years old are very close to 0, hence irrelevant.


For all datasets (D4, D5 and D6), the same process has been adopted. For the sake of documentation we will use D4 as an example.

1. Age groups have been aggregated, cleaning eventual unwanted values from source datasets 

In [18]:
pathD4 = "../data/srcDS/D4Pregnancy/cleanedDS/cleanedD4-2019-Pregnancy.csv"
live_births = read_csv(pathD4, keep_default_na=False,
            dtype= {
                "RESIDENCE_TERR":"string",
                "CITIZENSHIP_MOTHER": "string",
                "MOTHER_AGE" :"string",
                "OBS_VALUE" : "int64"
            })

live_births = live_births.query("MOTHER_AGE != 'Y25'") # create a df without data for Y25
live_births = live_births.query("RESIDENCE_TERR != 'ITD1'")
live_births = live_births.query("RESIDENCE_TERR != 'ITD2'") 
# take out the column with mother_age so I can sum all of the values per region
live_births = live_births[["RESIDENCE_TERR", "OBS_VALUE"]] 
live_births = live_births.groupby("RESIDENCE_TERR", as_index=False).sum()

2. Regional code and Region name have been added<br>
<center>In D4, that only had regional codes:</center>

In [19]:
ID_name_dict = { "ITC1" : "Piemonte",
                "ITC2" : "Valle d'Aosta / Vallée d'Aoste",
                "ITC3" : "Liguria",
                "ITC4" : "Lombardia",
                "ITDA" : "Trentino Alto Adige / Südtirol",
                "ITD3" : "Veneto",
                "ITD4" : "Friuli-Venezia Giulia",
                "ITD5" : "Emilia-Romagna",
                "ITE1" : "Toscana",
                "ITE2" : "Umbria",
                "ITE3" : "Marche",
                "ITE4" : "Lazio",
                "ITF1" : "Abruzzo",
                "ITF2" : "Molise",
                "ITF3" : "Campania",
                "ITF4" : "Puglia",
                "ITF5" : "Basilicata",
                "ITF6" : "Calabria",
                "ITG1" : "Sicilia",
                "ITG2" : "Sardegna"
                }


i = 0

# add NL Region name to dataset

region_name = []

for idx, row in live_births.iterrows():
    region_name.append(ID_name_dict[live_births.at[idx, "RESIDENCE_TERR"]])

live_births["Region"] = region_name #maybe try inserting at idx 1 ??



<center>2.2. In D5 and D5, that only had Region name:</center>

In [20]:
pathD5 = "../data/srcDS/D5Pregnancy/cleanedDS/cleanedD5-2019-Pregnancy.csv"
miscarriages = read_csv(pathD5, keep_default_na=False,
             dtype= {
                "Territorio":"string",
                "Classe di età": "string",
                "Value" : "int64"
            })

pathD6 = "../data/srcDS/D6Pregnancy/cleanedDS/cleanedD6-2019-Pregnancy.csv"
abortions = read_csv(pathD6, keep_default_na=False,
             dtype= {
                "Territorio dell'evento":"string",
                "Età e classe di età": "string",
                "Value" : "int64"
            })
            
region_code = []
region_name = []

for key, value in ID_name_dict.items():
    region_name.append(value)
    region_code.append(key)

DF_ID_name = DataFrame({"ITTER107" : Series(region_code,  dtype="string"),  "Region": Series(region_name,  dtype="string")})

miscarriages = merge(DF_ID_name, miscarriages, left_on="Region", right_on="Territorio")
miscarriages.drop(["Territorio"], axis=1, inplace=True)

abortions = merge(DF_ID_name, abortions, left_on="Region", right_on="Territorio dell'evento")
abortions.drop(["Territorio dell'evento"], axis=1, inplace=True)


3. If needed columns label have been renamed

In [24]:
live_births.rename(columns = {"RESIDENCE_TERR" : "ITTER107", "OBS_VALUE" : "Live_births"}, inplace=True)
miscarriages.rename(columns = {"Value" : "Miscarriages"}, inplace=True)
abortions.rename(columns = { "Value" : "Abortions"}, inplace=True)


4. A column <code>Time</code> specifying the year of the dataset has been added. <br><br>
For this purpose, a function <code>addyear</code> has been implemented. It takes in imput <code>pathyear</code> the path of the dataset of interest, and <code>df</code> the Dataframe object of the dataset of interest.<br>
If <code>Time</code> column is not already in the dataset, the function adds it, etracting the correct year from the name of the dataset found in the path. It returns <code>yr</code> the year of the dataset as a string


In [25]:
import re
def addyear(pathyear, df):
    if "Time" not in df.columns[0]:
        year2017 = re.search("2017", pathyear)
        year2018 = re.search("2018", pathyear)
        year2019 = re.search("2019", pathyear)

        years = [year2017, year2018, year2019]
        for el in years:
            if el is not None:
                yr = el.group()
                df["Time"] = yr
                return yr
        return False

yr = addyear(pathD4, live_births)

5. Finally, the resulting datasets are merged together based on Region's value and the total pregnancies are calculated summing each DS' value. They are exported as <b>csv</b> to create the final mashup dataset MD2.

In [26]:
PregnancyDS = merge(live_births, miscarriages, left_on= "ITTER107", right_on="ITTER107")
PregnancyDS = merge(PregnancyDS, abortions, left_on= "ITTER107", right_on="ITTER107", how="left") #how="left" is because I don'7 have lazio's value rn

total = PregnancyDS.sum(axis=1, numeric_only=True) # sums all pregnancies value per region
PregnancyDS["Total"] = total
PregnancyDS = PregnancyDS[["ITTER107", "Region", "Live_births", "Miscarriages", "Abortions", "Total", "Time"]]
PregnancyDS.to_csv("../data/mashupDS/MD2-ASS-" + yr + ".csv", index=False)

Find the visualisation of the 2017 mashup dataset with absolute values below as example.
All MD2 - Absolute values can be found in ` data > mashupDS ` <br><br>
Full code is available at `scripts > MASHUP.py `

In [29]:
md2_ass_2017 = read_csv("../data/mashupDS/MD2-ASS-2017.csv")
md2_ass_2017

,ITTER107,Region,Live_births,Miscarriages,Abortions,Total,Time
0,ITC1,Piemonte,5448,309,1670,7427,2017
1,ITC2,Valle d'Aosta / Vallée d'Aoste,142,9,46,197,2017
2,ITC3,Liguria,1794,92,664,2550,2017
3,ITC4,Lombardia,13468,599,3381,17448,2017
4,ITD3,Veneto,5944,287,1116,7347,2017
5,ITD4,Friuli-Venezia Giulia,1342,75,343,1760,2017
6,ITD5,Emilia-Romagna,6016,234,1644,7894,2017
7,ITDA,Trentino Alto Adige / Südtirol,1776,50,326,2152,2017
8,ITE1,Toscana,4392,221,1310,5923,2017
9,ITE2,Umbria,1056,41,275,1372,2017


#### MD2 - Percentage values

This dataset contains the <b>percentage values of total pregnancies, live births, spontaneous abortions and induced abortions</b> calculated over the total female population aged 15-25.

The following datasets have been used:<br><br>
<b>D1-D2 selected Population cleaned datasets</b><br>

* Population2017Selected_clean.csv
* Population2018Selected_clean.csv
* Population2019Selected_clean.csv

<b> MD2 with absolute values </b><br>

* MD2-ASS-2017.csv
* MD2-ASS-2018.csv
* MD2-ASS-2019.csv


1. Males values + Total values have been taken out from selected population datasets whereas Female values have been summed and grouped by region


In [31]:
# SELECTED POPULATION FILES
sel_pop17 = read_csv("../data/cleanDS/Population2017Selected_clean.csv")
# only keep "Females" column
sel_pop17 = sel_pop17.drop(sel_pop17[(sel_pop17.Sex == "Males") | (sel_pop17.Sex == "Total")].index)
# Put together ages and sum value under column "Population"
sel_pop17 = (sel_pop17.groupby(["Region code","Region"])["Population"].sum()).reset_index()
sel_pop17["Time"] = 2017
sel_pop17.rename(columns = {"Region code" : "ITTER107", "Population" : "Females"}, inplace=True)

def selectedPop(path):
    file = read_csv(path)
    file.drop(["Males","Population"],axis=1,inplace=True)
    file = (file.groupby(["ITTER107","Region"])["Females"].sum()).reset_index()
    return file    

pop18 = "../data/cleanDS/Population2018Selected_clean.csv"
pop19 = "../data/cleanDS/Population2019Selected_clean.csv"

sel_pop18 = selectedPop(pop18)
sel_pop18["Time"] = 2018   # adds the Time column
sel_pop19 = selectedPop(pop19)
sel_pop19["Time"] = 2019


2. A simple function <code>getPercentage</code> as been implemented. <br><br> It takes a <code>num</code> numerator and a <code>pop</code> denominator value and returns the result of the percentage equation.

In [32]:
def getPercentage(num,pop):
    return num * 100 / pop

3. Another function <code>percentages</code> has been implemented. <br><br> It takes in input the dataset <code>DStotal</code> of the total selected female population and the dataset <code>DSpartial</code> of the mashup datasets MD2 with pregnancies absolute values.<br><br>
This function transforms absolute values of <code>DSpartial</code> in percentage values and export the result dataset in <b>csv</b> format

In [35]:
# create empty lists to fill with % values
Total_perc = []
Live_births_perc = []
Miscarriages_perc = []
Abortions_perc = []

def percentages(DStotal, DSpartial):
    # merge total DS with partial DS so it's then possible to iterate over rows
    mergedDF = merge(DStotal, DSpartial, left_on="ITTER107", right_on="ITTER107", how="left")
    mergedDF = mergedDF[["ITTER107", "Region_x", "Females", "Live_births", "Miscarriages", "Abortions", "Total"]]

    for idx, row in mergedDF.iterrows():
        year = str(DStotal.at[idx, "Time"]) # save year value to add later as a column

        # calculate percentages for each value
        p_tot = getPercentage((mergedDF.at[idx, "Total"]), (mergedDF.at[idx, "Females"]))
        p_lb = getPercentage((mergedDF.at[idx, "Live_births"]), (mergedDF.at[idx, "Females"]))
        p_m = getPercentage((mergedDF.at[idx, "Miscarriages"]), (mergedDF.at[idx, "Females"]))
        p_a = getPercentage((mergedDF.at[idx, "Abortions"]), (mergedDF.at[idx, "Females"]))

        Total_perc.append(p_tot)
        Live_births_perc.append(p_lb)
        Miscarriages_perc.append(p_m)
        Abortions_perc.append(p_a)

    # change absolute values with percentage values
    DSpartial["Live_births"] = Live_births_perc
    DSpartial["Miscarriages"] = Miscarriages_perc
    DSpartial["Abortions"] = Abortions_perc
    DSpartial["Total"] = Total_perc
    DSpartial["Time"] = year

    DSpartial.to_csv("../data/mashupDS/MD2-PERC-" + year + ".csv", index=False)
    return True


Find the visualisation of the 2017 mashup dataset with percentage values below as example.
All MD2 - Absolute values can be found in ` data > mashupDS ` <br><br>
Full code is available at `scripts > MASHUP.py `

In [36]:
md2_perc_2017 = read_csv("../data/mashupDS/MD2-PERC-2017.csv")
md2_perc_2017

,ITTER107,Region,Live_births,Miscarriages,Abortions,Total,Time
0,ITC1,Piemonte,2.657263,0.150715,0.814543,3.622520,2017
1,ITC2,Valle d'Aosta / Vallée d'Aoste,2.308943,0.146341,0.747967,3.203252,2017
2,ITC3,Liguria,2.609758,0.133834,0.965931,3.709523,2017
3,ITC4,Lombardia,2.733171,0.121560,0.686134,3.540865,2017
4,ITD3,Veneto,2.409824,0.116356,0.452450,2.978630,2017
5,ITD4,Friuli-Venezia Giulia,2.431512,0.135889,0.621467,3.188868,2017
6,ITD5,Emilia-Romagna,2.912133,0.113271,0.795802,3.821206,2017
7,ITDA,Trentino Alto Adige / Südtirol,2.891472,0.081404,0.530754,3.503631,2017
8,ITE1,Toscana,2.566290,0.129133,0.765446,3.460869,2017
9,ITE2,Umbria,2.557644,0.099302,0.666053,3.322999,2017


### MD3 - (Higher) education rates in young women in each region
As the source dataset D7 did not have, with absolute values, the subdivision in regions (but only had data regarding the whole national territory of Italy), we had to use the "percentage values".
The percentage is calculated on the total population of 18-24 years old in each region, without distinctions in terms of gender.

However, given the assumptions of the project, we had to focus on the female population of this age range and we did so by employing the following proportion:
* % early leavers : total 18-24 = [% early leavers F] : total 18-24 F

Where:
1. total 18-24 -> data to be gathered from the general population datasets (D1,D2), filtering them with ages 18-24
2. total 18-24 F -> data to be gathered from the selected population datasets (D1, D2), filtering them with ages 18-24
3.  % early leavers -> data contained in the source dataset D7
4. [% early leavers F] -> data to be gathered following the proportion

#### Step 1: Total population 18-24 for each region

In [ ]:
# === A) TOTAL 18-24
# Get data for ages 18-24 from the general population datasets
def selectAge(df, age1, age2):
    df = df.drop(df[(df.Age == "100 and over") | (df.Age == "Total")].index)
    df["Age"] = df["Age"].astype('int')
    young_df = df.loc[(df["Age"] >= age1) & (df["Age"] <= age2)]
    return young_df


def young_generalPop(df, age1, age2):
    young_df = selectAge(df, age1, age2)
    young_df = (df.groupby(["ITTER107", "Region"])
                ["Population"].sum()).reset_index()
    return young_df


# --- 2017
pop17 = read_csv("../data/cleanDS/Population2017General_clean.csv")
# Delete gender distinction
pop17 = pop17.drop(pop17[(pop17.Sex == "Males") |
                   (pop17.Sex == "Females")].index)
# Fixing the "Age" column (changing it to int values) to allow for a better condition for the creation of young_pop17
young_pop17 = selectAge(pop17, 18, 24)
young_pop17 = (pop17.groupby(["Region code", "Region"])[
               "Population"].sum()).reset_index()
# Adding columns for easier identification
young_pop17["Time"] = 2017
young_pop17["Age range"] = "18-24"

# --- 2018
pop18 = read_csv("../data/cleanDS/Population2018General_clean.csv")
pop18.drop(["Males", "Females"], axis=1, inplace=True)
young_pop18 = young_generalPop(pop18, 18, 24)
young_pop18["Time"] = 2018
young_pop18["Age range"] = "18-24"

# --- 2019
pop19 = read_csv("../data/cleanDS/Population2019General_clean.csv")
pop19.drop(["Males", "Females"], axis=1, inplace=True)
young_pop19 = young_generalPop(pop19, 18, 24)
young_pop19["Time"] = 2019
young_pop19["Age range"] = "18-24"


#### Step 2: Total female population 18-24 for each region


In [ ]:
# === B) TOTAL 18-24 F
# Same as before, just DO NOT delete the gender distinction
# --- 2017
f_pop17 = read_csv("../data/cleanDS/Population2017General_clean.csv")
f_pop17 = f_pop17.drop(
    f_pop17[(f_pop17.Sex == "Males") | (f_pop17.Sex == "Total")].index)
# Fixing the "Age" column (changing it to int values) to allow for a better condition for the creation of youngF_pop17
youngF_pop17 = selectAge(f_pop17, 18, 24)
youngF_pop17 = (f_pop17.groupby(["Region code", "Region"])[
                "Population"].sum()).reset_index()
# Adding columns for easier identification
youngF_pop17["Time"] = 2017
youngF_pop17["Age range"] = "18-24"
youngF_pop17["Gender"] = "Females"

# --- 2018
f_pop18 = read_csv("../data/cleanDS/Population2018General_clean.csv")
youngF_pop18 = young_generalPop(f_pop18, 18, 24)
youngF_pop18["Time"] = 2018
youngF_pop18["Age range"] = "18-24"
youngF_pop18["Gender"] = "Females"

# --- 2019
f_pop19 = read_csv("../data/cleanDS/Population2019General_clean.csv")
youngF_pop19 = young_generalPop(f_pop19, 18, 24)
youngF_pop19["Time"] = 2019
youngF_pop19["Age range"] = "18-24"
youngF_pop19["Gender"] = "Females"

#### Step 3: Percentage of early leavers for each region

In [ ]:
d7_clean = read_csv("../data/cleanDS/D7_clean.csv")
d7_2017 = d7_clean.drop(d7_clean[(d7_clean.TIME == 2018) | (d7_clean.TIME == 2019)].index)
d7_2018 = d7_clean.drop(d7_clean[(d7_clean.TIME == 2017) | (d7_clean.TIME == 2019)].index)
d7_2019 = d7_clean.drop(d7_clean[(d7_clean.TIME == 2017) | (d7_clean.TIME == 2018)].index)


#### Step 4: Percentage of female early leavers for each region

(this last step has been commented out after a first run in order to avoid interfeering with the rest of the code/project)

In [ ]:
# === C) % EARLY LEAVERS F (PROPORTION)
# --- 2017
MD3_2017 = (merge(young_pop17,d7_2017,left_on="Region code",right_on="ITTER107")).drop(["TIME","Territory"],axis=1)
MD3_2017 = (merge(youngF_pop17,MD3_2017,left_on="Region code",right_on="ITTER107",suffixes=('_GENERAL','_FEMALE'))).drop(["Time_FEMALE","Age range_FEMALE","ITTER107","Region code_FEMALE","Region_FEMALE"],axis=1)
for idx,row in MD3_2017.iterrows():
    result = (row["Value"] * row["Population_FEMALE"])/(row["Population_GENERAL"])
    MD3_2017.loc[idx,"Female Early Leavers"] = result
#MD3_2017.to_csv("../data/mashupDS/MD3_17.csv")


# --- 2018
MD3_2018 = (merge(young_pop18,d7_2018,left_on="ITTER107",right_on="ITTER107")).drop(["TIME","Territory"],axis=1)
MD3_2018 = (merge(youngF_pop18,MD3_2018,left_on="ITTER107",right_on="ITTER107",suffixes=('_GENERAL','_FEMALE'))).drop(["Time_FEMALE","Age range_FEMALE","Region_FEMALE"],axis=1)
for idx,row in MD3_2018.iterrows():
    result = (row["Value"] * row["Population_FEMALE"])/(row["Population_GENERAL"])
    MD3_2018.loc[idx,"Female Early Leavers"] = result
#MD3_2018.to_csv("../data/mashupDS/MD3_18.csv")


# --- 2019
MD3_2019 = (merge(young_pop19,d7_2019,left_on="ITTER107",right_on="ITTER107")).drop(["TIME","Territory"],axis=1)
MD3_2019 = (merge(youngF_pop19,MD3_2019,left_on="ITTER107",right_on="ITTER107",suffixes=('_GENERAL','_FEMALE'))).drop(["Time_FEMALE","Age range_FEMALE","Region_FEMALE"],axis=1)
for idx,row in MD3_2019.iterrows():
    result = (row["Value"] * row["Population_FEMALE"])/(row["Population_GENERAL"])
    MD3_2019.loc[idx,"Female Early Leavers"] = result
#MD3_2019.to_csv("../data/mashupDS/MD3_19.csv")
